In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.backends.mps.is_available():
    device = torch.device('mps')

print(f"Using device: {device}")

Using device: cuda


In [5]:
BATCH_SIZE = 128
IMG_SIZE = 224
DATA_DIR = r'F:\WIDS-5.0\data\plantvillage dataset\color' # <--- CHANGE THIS

# ImageNet statistics for normalization
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std) # Critical for Transfer Learning
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std)
])

In [6]:
full_dataset = datasets.ImageFolder(root=DATA_DIR)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transforms
val_dataset.dataset.transform = val_transforms

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

class_names = full_dataset.classes

In [7]:
# Load ResNet18 with default weights (ImageNet)
model_tl = models.resnet18(weights='DEFAULT')

# Freeze the backbone
for param in model_tl.parameters():
    param.requires_grad = False

# Replace the final fully connected layer (fc)
# ResNet18 input to fc is 512
num_ftrs = model_tl.fc.in_features
model_tl.fc = nn.Linear(num_ftrs, len(class_names))

model_tl = model_tl.to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\ruchi/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


In [8]:
criterion = nn.CrossEntropyLoss()

# Note: We pass the optimizer as an argument so we can switch it later
def train_one_epoch(model, loader, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    return running_loss / len(loader), 100 * correct / total

def evaluate(model, loader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return running_loss / len(loader), 100 * correct / total

In [9]:
# Create optimizer only for the parameters that require gradients (the new head)
optimizer_head = optim.Adam(filter(lambda p: p.requires_grad, model_tl.parameters()), lr=0.001)

print("🚀 Starting Feature Extraction Training...")
for epoch in range(5):
    train_loss, train_acc = train_one_epoch(model_tl, train_loader, optimizer_head)
    val_loss, val_acc = evaluate(model_tl, val_loader)
    print(f"Epoch {epoch+1}/5 | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

🚀 Starting Feature Extraction Training...
Epoch 1/5 | Train Acc: 82.82% | Val Acc: 93.39%
Epoch 2/5 | Train Acc: 94.10% | Val Acc: 94.63%
Epoch 3/5 | Train Acc: 95.35% | Val Acc: 95.40%
Epoch 4/5 | Train Acc: 96.04% | Val Acc: 95.76%
Epoch 5/5 | Train Acc: 96.48% | Val Acc: 96.24%


In [10]:
print("🔥 Unfreezing backbone for Fine-Tuning...")

# Unfreeze all layers
for param in model_tl.parameters():
    param.requires_grad = True

# Use a much smaller learning rate (1e-4) to avoid destroying learned features
optimizer_ft = optim.Adam(model_tl.parameters(), lr=1e-4)

# Train for a few more epochs with the new optimizer
for epoch in range(5):
    train_loss, train_acc = train_one_epoch(model_tl, train_loader, optimizer_ft)
    val_loss, val_acc = evaluate(model_tl, val_loader)
    print(f"Fine-Tune Epoch {epoch+1}/5 | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

🔥 Unfreezing backbone for Fine-Tuning...
Fine-Tune Epoch 1/5 | Train Acc: 98.23% | Val Acc: 99.21%
Fine-Tune Epoch 2/5 | Train Acc: 99.80% | Val Acc: 99.13%
Fine-Tune Epoch 3/5 | Train Acc: 99.93% | Val Acc: 99.45%
Fine-Tune Epoch 4/5 | Train Acc: 99.97% | Val Acc: 99.59%
Fine-Tune Epoch 5/5 | Train Acc: 99.99% | Val Acc: 99.62%
